In [1]:
import pandas as pd
from pandas.io import gbq


df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRhcAWkWXIjp2XVAsnTLw13QGg6Ot9D_HBf_FMCA42qIWf034T8oKOgV6cTBJS29tfJRPHPyQ4DQJ6s/pub?gid=1990926458&single=true&output=csv")

df = df.rename({"Timestamp":"last_record"}, axis=1)

df["Date Leads Acquired"] = df["Date Leads Acquired"].astype('datetime64[ns]')
df["Delegation Date"] = df["Delegation Date"].astype('datetime64[ns]')
df["Main Activity Start Date"] = df["Main Activity Start Date"].astype('datetime64[ns]')
df["Main Activity End Date"] = df["Main Activity End Date"].astype('datetime64[ns]')
df["Final Report Submitted Date"] = df["Final Report Submitted Date"].astype('datetime64[ns]')
df["Project Archived Date"] = df["Project Archived Date"].astype('datetime64[ns]')

df_drop = df.drop(["Project Name", "Date Leads Acquired", 
                    "Delegation Date", "Main Activity Start Date",
                    "Main Activity End Date", "Final Report Submitted Date", 
                    "Project Archived Date"], axis=1)

def phase(start, finish, phase):
    df_phase = df[["Project Name", start, finish]]
    df_phase = pd.concat([df_phase, df_drop], axis=1)
    df_phase = df_phase.dropna(subset=["Project Name"])
    df_phase["Phase"] = phase
    df_phase.rename(columns = {"Project Name":"Project", start:"Start", finish:"Finish"}, inplace=True)
    df_phase["Start"] = pd.to_datetime(df_phase["Start"])
    df_phase["Finish"] = pd.to_datetime(df_phase["Finish"])
    return df_phase

#Create Phase Columns
df_development = phase(start="Date Leads Acquired", finish="Delegation Date", phase="Development")
df_preparation = phase(start="Delegation Date", finish="Main Activity Start Date", phase="Preparation")
df_active = phase(start="Main Activity Start Date", finish="Main Activity End Date", phase="Active")
df_reporting = phase(start="Main Activity End Date", finish="Final Report Submitted Date", phase="Reporting")
df_closing = phase(start="Final Report Submitted Date", finish="Project Archived Date", phase="Closing")
df_gantt = pd.concat(
                [df_development, df_preparation, df_active, df_reporting, df_closing], 
                ignore_index=True)

df_gantt["timestamp"] = pd.Timestamp('now')

#cleaning columns
replace = {
    ' ' : '_',
    '-' : '_',
    '.' : '_',
    '?' : '',
    '[' : '',
    ']' : '',
    '(' : '',
    ')' : ''
}

def clean_col (col, clean_col):
  col.columns = col.columns.str.lower()
  for i, j in clean_col.items():
    col.columns = col.columns.str.replace(i, j)
  return col

df_2 = clean_col(df_gantt, replace)


pd.set_option('display.max_columns', None)
df_2.head(1)

/tmp/ipykernel_16938/1244764655.py:58: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  col.columns = col.columns.str.replace(i, j)
/tmp/ipykernel_16938/1244764655.py:58: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  col.columns = col.columns.str.replace(i, j)
/tmp/ipykernel_16938/1244764655.py:58: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  col.columns = col.columns.str.replace(i, j)
/tmp/ipykernel_16938/1244764655.py:58: FutureWarning: The default value of regex will change from True to False in a future version. In add

,project,start,finish,project_id,preparation_start_date,ready_to_deploy_date,project_end_date,service_type,in_house_pic,project_objectives,project_key_results,project_main_activities,project_value,proposed_cogs,planned_project_start_date,planned_ready_to_deploy_date,planned_main_activity_start_date,planned_main_activity_end_date,planned_final_report_submitted_date,planned_project_end_date,planned_project_archived_date,proposed_nsm_projection,planned_project_strategic_index_profit_margin,planned_project_strategic_index_replicability_to_market,planned_project_strategic_index_influencing_decision_maker,planned_project_strategic_index_contributes_to_swm_intelligence,planned_project_strategic_index_catalyzing_behaviour_change,planned_project_strategic_index_quantity_of_beneficiaries,planned_project_strategic_index_ease_of_implementation,planned_project_strategic_index_supports_nsm_financially,planned_project_strategic_index_supports_nsm_strategically,email_address,the_actual_data_is_available,actual_project_start_date,actual_ready_to_deploy_date,actual_main_activity_start_date,actual_main_activity_end_date,actual_final_report_submitted_date,actual_project_end_date,actual_project_archived_date,actual_nsm_projection,actual_project_cogs,last_record,email_address_1,client_organization,new_client_organization_name,project_name_1,w4c_pic,client_organization_type,new_client_organization_type,client_organization_industry,new_client_organization_industry,client_organization_size,client_organization_location,client_pic,client_pic_position,client_phone,client_email,lead_status,service_type_1,service_topic,new_service_topic,chance,date_project_decided,date_proposal_sent,number_of_proposal_revision,operational_value,non_ss_operational_value,in_house_consultant_involved,total_personnel_value_albert,total_billed_days_albert,other_in_house_consultant,total_personnel_value_anin,total_billed_days_anin,other_in_house_consultant_1,total_personnel_value_april,total_billed_days_april,other_in_house_consultant_2,total_personnel_value_elma,total_billed_days_elma,other_in_house_consultant_3,total_personnel_value_fitria,total_billed_days_fitria,other_in_house_consultant_4,total_personnel_value_ica,total_billed_days_ica,other_in_house_consultant_5,total_personnel_value_ifa,total_billed_days_ifa,other_in_house_consultant_6,total_personnel_value_khai,total_billed_days_khai,other_in_house_consultant_7,total_personnel_value_prita,total_billed_days_prita,other_in_house_consultant_8,total_personnel_value_saka,total_billed_days_saka,other_in_house_consultant_9,total_personnel_value_tantin,total_billed_days_tantin,other_in_house_consultant_10,number_of_other_in_house_consultants,names_of_in_house_consultant,total_personnel_value_other,total_billed_days_other,other_in_house_consultant_11,total_analyst_personnel_value,total_analyst_billed_days,total_associate_personnel_value,total_associate_billed_days,total_consultant_personnel_value,total_consultant_billed_days,other_in_house_consultant_12,proposed_cogs_1,proposed_project_start_date,proposed_project_end_date,nsm_projection,project_strategic_index_projection_profit_margin,project_strategic_index_projection_repicability_to_market,project_strategic_index_projection_influencing_decision_maker,project_strategic_index_projection_contributes_to_swm_intelligence,project_strategic_index_projection_catalyzing_behaviour_change,project_strategic_index_projection_quantity_of_beneficiaries,project_strategic_index_projection_ease_of_implementation,project_strategic_index_projection_supports_nsm_financially,project_strategic_index_projection_supports_nsm_strategically,delegation_date_1,contract_status,contract_signature_date,terms_of_payment_quantity,term_of_payment_value,milestone,estimated_date_of_milestone_delivery,final_term_of_payment,term_of_payment_value_1,milestone_1,estimated_date_of_milestone_delivery_1,final_term_of_payment_1,term_of_payment_value_2,milestone_2,estimated_date_of_milestone_delivery_2,final_term_of_payment